### Ваша задача — реализовать алгоритм случайного леса для задачи классификации и применить его к набору данных. 

Шаги выполнения задания:

Импортируйте необходимые библиотеки: numpy, pandas, sklearn.

Загрузите набор данных для задачи классификации. Вы можете использовать любой доступный набор данных или выбрать один из популярных, таких как Iris, Wine или MNIST.

Проведите предварительную обработку данных, включая масштабирование и разделение на обучающую и тестовую выборки.

Реализуйте алгоритм случайного леса. Включите в него функции для построения деревьев решений, выбора случайных признаков и голосования для принятия решений.

Обучите вашу модель случайного леса на обучающей выборке.

Оцените производительность модели на тестовой выборке, используя метрики классификации, такие как точность, полнота и F1-мера.

Проведите сравнение результатов вашей модели со стандартной реализацией случайного леса из библиотеки scikit-learn.

## РЕШЕНИЕ 
Импортируем необходимые библиотеки для работы.

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
# 2. Загружаем набор данных Iris
iris = load_iris()
X = iris.data
y = iris.target

In [3]:
# Предварительная обработка данных
# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### Реализация алгоритма случайного леса

Теперь создадим класс для случайного леса, который будет включать создание деревьев и процедуру голосования.

#### Обучение модели случайного леса

In [4]:
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        unique_classes = np.unique(y)
        
        # Если все классы одинаковые, возвращаем их
        if len(unique_classes) == 1:
            return unique_classes[0]
        
        # Если достигли максимальной глубины, возвращаем моду
        if self.max_depth is not None and depth >= self.max_depth:
            return self._majority_class(y)

        # Поиск наилучшего разбиения
        best_gain = -1
        best_split = None
        best_left_indices = None
        best_right_indices = None

        for feature_index in range(num_features):  # Перебор всех признаков
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:  # Перебор всех возможных значений разбиения
                left_indices = np.where(X[:, feature_index] <= threshold)[0]
                right_indices = np.where(X[:, feature_index] > threshold)[0]

                if len(left_indices) > 0 and len(right_indices) > 0:
                    gain = self._information_gain(y, left_indices, right_indices)

                    if gain > best_gain:
                        best_gain = gain
                        best_split = (feature_index, threshold)
                        best_left_indices = left_indices
                        best_right_indices = right_indices

        if best_gain == -1:
            return self._majority_class(y)

        # Рекурсивное построение поддеревьев
        left_tree = self._build_tree(X[best_left_indices], y[best_left_indices], depth + 1)
        right_tree = self._build_tree(X[best_right_indices], y[best_right_indices], depth + 1)

        return (best_split, left_tree, right_tree)

    def _information_gain(self, y, left_indices, right_indices):
        p = float(len(left_indices)) / len(y)
        return self._gini(y) - p * self._gini(y[left_indices]) - (1 - p) * self._gini(y[right_indices])

    def _gini(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities ** 2)

    def _majority_class(self, y):
        return np.bincount(y).argmax()

    def predict(self, X):
        predictions = [self._predict_sample(sample, self.tree) for sample in X]
        return np.array(predictions)

    def _predict_sample(self, sample, tree):
        if isinstance(tree, tuple):  # Если это узел дерева
            feature_index, threshold = tree[0]
            if sample[feature_index] <= threshold:
                return self._predict_sample(sample, tree[1])
            else:
                return self._predict_sample(sample, tree[2])
        else:
            return tree  # Если это лист дерева

In [5]:
class RandomForest:
    def __init__(self, n_trees=100, max_depth=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        for _ in range(self.n_trees):
            # Бутстрэппинг: выбираем случайные подмножества данных
            indices = np.random.choice(len(X), len(X), replace=True)
            X_subsample, y_subsample = X[indices], y[indices]
            tree = DecisionTree(max_depth=self.max_depth)
            tree.fit(X_subsample, y_subsample)
            self.trees.append(tree)

    def predict(self, X):
        # Получение предсказаний от каждого дерева
        predictions = np.array([tree.predict(X) for tree in self.trees])
        # Голосование
        return np.array([np.bincount(p).argmax() for p in predictions.T])

In [6]:
# Обучение модели случайного леса
model = RandomForest(n_trees=100, max_depth=5)
model.fit(X_train, y_train)

In [7]:
# Оценка производительности модели
y_pred = model.predict(X_test)

In [8]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print(f'Точность: {accuracy:.4f}')
print(f'Точность: {precision:.4f}')
print(f'Полнота: {recall:.4f}')
print(f'F1-мера: {f1:.4f}')

Точность: 1.0000
Точность: 1.0000
Полнота: 1.0000
F1-мера: 1.0000


Объяснение:
#### Класс DecisionTree:
- Метод fit обучает дерево, создавая рекурсивно его структуру, выбирая наилучшее разбиение по критерию Джини.
- Метод _build_tree строит дерево.
- Метод _predict_sample предсказывает класс для отдельного наблюдения.

#### Класс RandomForest:
- Метод fit создает несколько деревьев, обучая каждое из них на бутстрэппированной выборке.
- Метод predict собирает предсказания всех деревьев и использует голосование для определения итогового класса.

#### Обучение и оценка:
- Набор данных Iris загружается, разбивается на обучающую и тестовую выборки.
- Модель случайного леса обучается и оценивается с использованием метрик точности, точности, полноты и F1-меры.

### Сравнение сo стандартной реализацией

In [9]:
model_sklearn = RandomForestClassifier(n_estimators=100, random_state=42)
model_sklearn.fit(X_train, y_train)
y_pred_sklearn = model_sklearn.predict(X_test)

In [10]:
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
precision_sklearn = precision_score(y_test, y_pred_sklearn, average='macro')
recall_sklearn = recall_score(y_test, y_pred_sklearn, average='macro')
f1_sklearn = f1_score(y_test, y_pred_sklearn, average='macro')

print(f'Точность (sklearn): {accuracy_sklearn:.4f}')
print(f'Точность (sklearn): {precision_sklearn:.4f}')
print(f'Полнота (sklearn): {recall_sklearn:.4f}')
print(f'F1-мера (sklearn): {f1_sklearn:.4f}')

Точность (sklearn): 1.0000
Точность (sklearn): 1.0000
Полнота (sklearn): 1.0000
F1-мера (sklearn): 1.0000


Результаты в обоих случаях оказались одинаковыми, что можно объяснить небольшим количеством признаков в датасете и их высокой корреляцией. Для практических целей удобнее и быстрее использовать стандартную реализацию.

Метрики модели демонстрируют отличную производительность.

- Точность (Precision) 1.0000 показывает, что модель правильно классифицирует все предсказанные положительные случаи.
- Полнота (Recall) 1.0000 указывает на то, что модель выявляет все истинные положительные случаи. 
- F1-мера 1.0000 — это гармоническое среднее точности и полноты, подтверждающее идеальную сбалансированную производительность модели.

Такой идеальный результат редко встречается в реальных задачах. Важно помнить, что это может быть обусловлено следующими факторами:

- Недостатком данных: модель может быть обучена на слишком простом наборе данных, где классы легко различимы.
- Переобучением: модель может переобучиться на тренировочных данных и плохо обобщать на новые данные.